In [9]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

class FertilizerStandardizer:
    def __init__(self):
        # Liste des engrais ICV
        self.icv_fertilizer_list = ["Solution Liquide N 39"]
        
        # Masses volumiques 
        self.default_mineral_density = 1.325  # kg/L pour minéraux liquides
        self.organic_solid_density = 1000     # kg/m³ pour organiques solides
        self.organic_liquid_density = 1.0     # kg/L pour organiques liquides

        # Mapping des types d'intervention vers type_engrais
        self.type_mapping = {
            # Mesparcelles
            "Fertilisation et amendement organique": "engrais organiques",
            "Fertilisation et amendement mineral - foliaire inclus": "engrais mineraux",
            # SMAG
            "Fertilisation organique": "engrais organiques",
            "Fertilisation minérale": "engrais mineraux"
        }

 
    def standardize_fertilizer_name(self, name):
        if pd.isna(name):
            return name
        # Forcer la conversion en string même si c'est une date
        if isinstance(name, (pd.Timestamp, datetime)):
            name = name.strftime('%d-%m-%Y')
        name = str(name).upper()
        name = re.sub(r'[^\w\s\-]', ' ', name)  # Garder les tirets
        name = re.sub(r'\s+', ' ', name)
        return name.strip()
    def is_in_icv_list(self, fertilizer_name):
        std_name = self.standardize_fertilizer_name(fertilizer_name)
        std_icv_list = [self.standardize_fertilizer_name(name) for name in self.icv_fertilizer_list]
        return std_name in std_icv_list

    def standardize_type_engrais(self, intervention_type):
        if pd.isna(intervention_type):
            return "engrais mineraux"
        type_std = self.type_mapping.get(intervention_type)
        if type_std is None:
            print(f"Avertissement: Type d'intervention non reconnu: {intervention_type}")
            return "engrais mineraux"
        return type_std

    def convert_to_kg(self, row):
        try:
            if pd.isna(row.get('unite_intrant_intervention')) or pd.isna(row.get('quantite_totale')):
                return np.nan
                
            unite = str(row.get('unite_intrant_intervention', '')).upper()
            type_engrais = str(row.get('type_engrais', '')).lower()
            quantite = float(row.get('quantite_totale', 0))
            
            if unite == 'KG':
                return quantite
            elif unite == 'T':
                return quantite * 1000
            elif unite == 'L':
                if 'organique' in type_engrais:
                    return quantite * self.organic_liquid_density
                else:
                    return quantite * self.default_mineral_density
            elif unite == 'M3':
                if 'organique' in type_engrais:
                    return quantite * self.organic_solid_density
                else:
                    return quantite * self.default_mineral_density * 1000
            else:
                print(f"Avertissement: Unité non reconnue: {unite} pour l'engrais {row.get('libelle', 'Unknown')}")
                return np.nan
        except Exception as e:
            print(f"Erreur lors de la conversion pour l'engrais {row.get('libelle', 'Unknown')}: {str(e)}")
            return np.nan

    def convert_dose_to_kg_ha(self, row):
        try:
            if pd.isna(row.get('unite')) or pd.isna(row.get('dose')):
                return np.nan
                
            unite = str(row['unite']).upper()
            if unite == 'KG/HA':
                return row['dose']
            elif unite == 'T/HA':
                return row['dose'] * 1000
            elif unite == 'L/HA':
                if row['type_engrais'] == 'engrais organiques':
                    return row['dose'] * self.organic_liquid_density
                else:
                    return row['dose'] * self.default_mineral_density
            elif unite == 'M3/HA':
                if row['type_engrais'] == 'engrais organiques':
                    return row['dose'] * self.organic_solid_density
                else:
                    return row['dose'] * self.default_mineral_density * 1000
            else:
                print(f"Avertissement: Unité non reconnue: {unite} pour l'engrais {row['libelle']}")
                return np.nan
        except Exception as e:
            print(f"Erreur lors de la conversion pour l'engrais {row['libelle']}: {str(e)}")
            return np.nan

    def calculate_nutrients(self, row):
        try:
            if not row['in_icv_list']:
                if row['type_engrais'] == 'engrais organiques':
                    volume_m3 = row['quantite_totale'] if row['unite_intrant_intervention'].upper() == 'M3' \
                        else row['quantite_kg'] / self.organic_solid_density
                    
                    row['N_kg_ha'] = row['fertilisant.composition.n_total'] * volume_m3 / row['surface_travaillee_ha']
                    row['P_kg_ha'] = row['fertilisant.composition.p'] * volume_m3 / row['surface_travaillee_ha']
                    row['K_kg_ha'] = row['fertilisant.composition.k'] * volume_m3 / row['surface_travaillee_ha']
                else:
                    row['N_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.n_total'] / 100
                    row['P_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.p'] / 100
                    row['K_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.k'] / 100
            return row
        except Exception as e:
            print(f"Erreur lors du calcul des nutriments pour l'engrais {row['libelle']}: {str(e)}")
            return row
        
    def standardize_mesparcelles(self, data):
        """Standardise les données MesParcelles"""
        print("Standardisation des données MesParcelles...")
        
        # Filtrer les fertilisations
        fertilisation_types = [
            "Fertilisation et amendement organique",
            "Fertilisation et amendement mineral - foliaire inclus"
        ]
        df = data[data['type_intervention.libelle'].isin(fertilisation_types)].copy()
        
        # Vérification de la colonne culture
        if 'culture.libelle' in df.columns:
            df = df.rename(columns={'culture.libelle': 'culture'})
            print("Colonne culture.libelle trouvée et renommée en 'culture'")
            print(f"Cultures uniques dans MesParcelles: {df['culture'].unique()}")
        else:
            print("ATTENTION: culture.libelle non trouvée dans MesParcelles!")
            print("Colonnes disponibles:", df.columns.tolist())
            return None
        
        # Standardisation
        df['type_engrais'] = df['type_intervention.libelle'].apply(self.standardize_type_engrais)
        df['in_icv_list'] = df['libelle'].apply(self.is_in_icv_list)
        df['quantite_kg'] = df.apply(self.convert_to_kg, axis=1)
        df['dose_kg_ha'] = df['quantite_kg'] / df['surface_travaillee_ha']
        df = df.apply(self.calculate_nutrients, axis=1)
        
        print(f"\nAprès standardisation MesParcelles:")
        print(f"Nombre d'observations: {len(df)}")
        print(f"Nombre d'exploitations: {df['siret_exploitation'].nunique()}")
        print(f"Nombre de cultures: {df['culture'].nunique()}")
        
        return self._aggregate_by_exploitation_and_culture(df)

    def standardize_smag(self, data, cultures_data):
        """Standardise les données SMAG"""
        print("\nStandardisation des données SMAG...")
        
        # Filtrer les fertilisations
        fertilisation_types = ["Fertilisation organique", "Fertilisation minérale"]
        df = data[data["Type d'intervention"].isin(fertilisation_types)].copy()
        
        # Fusionner avec les données de culture
        print(f"\nAvant fusion - Nombre de lignes: {len(df)}")
        df = pd.merge(
            df,
            cultures_data[['Code edi parcelle', 'Culture']],
            on='Code edi parcelle',
            how='left'
        )
        
        print(f"Après fusion - Nombre de lignes: {len(df)}")
        print(f"Lignes avec culture non-définie: {df['Culture'].isna().sum()}")
        
        # Renommage des colonnes
        df = df.rename(columns={
            'Intrant': 'libelle',
            'Unité': 'unite',
            'Dose': 'dose',
            'Surface intervention sur parcelle': 'surface_travaillee_ha',
            'N': 'N_kg_ha',
            'P': 'P_kg_ha',
            'K': 'K_kg_ha',
            'SIRET': 'siret_exploitation',
            'Code edi parcelle': 'uuid_parcelle',
            'Culture': 'culture'
        })
        
        # Standardisation
        df['type_engrais'] = df["Type d'intervention"].apply(self.standardize_type_engrais)
        df['in_icv_list'] = df['libelle'].apply(self.is_in_icv_list)
        df.loc[df['in_icv_list'], 'dose_kg_ha'] = df[df['in_icv_list']].apply(
            self.convert_dose_to_kg_ha, axis=1)
            
        print(f"\nAprès standardisation SMAG:")
        print(f"Nombre d'observations: {len(df)}")
        print(f"Nombre d'exploitations: {df['siret_exploitation'].nunique()}")
        print(f"Nombre de cultures: {df['culture'].nunique()}")
        
        return self._aggregate_by_exploitation_and_culture(df)

    def _aggregate_by_exploitation_and_culture(self, df):
        """Agrège les données par exploitation et par culture avec pondération par surface"""
        print("\nAgrégation par exploitation et culture...")
        
        def weighted_average(group):
            result = {}
            weights = group['surface_travaillee_ha']
            
            # Surface totale pour cette combinaison exploitation-culture
            result['surface_totale_ha'] = weights.sum()
            
            # Liste des engrais (conversion forcée en string)
            result['engrais_utilises'] = ', '.join(sorted(
                [str(x) for x in group['libelle'].unique()]
            ))
            
            # Pour les engrais ICV
            if 'dose_kg_ha' in group.columns:
                icv_mask = group['in_icv_list']
                if icv_mask.any():
                    for fertilizer in group.loc[icv_mask, 'libelle'].unique():
                        if pd.notnull(fertilizer):
                            fertilizer_str = str(fertilizer)
                            mask = icv_mask & (group['libelle'].astype(str) == fertilizer_str)
                            if mask.any():
                                result[f"{fertilizer_str}_dose_kg_ha"] = np.average(
                                    group.loc[mask, 'dose_kg_ha'],
                                    weights=group.loc[mask, 'surface_travaillee_ha']
                                )
            
            # Pour les nutriments (non-ICV)
            non_icv_mask = ~group['in_icv_list']
            if non_icv_mask.any():
                result['engrais_non_icv'] = ', '.join(sorted(
                    [str(x) for x in group.loc[non_icv_mask, 'libelle'].unique()]
                ))
                
                for nutrient in ['N_kg_ha', 'P_kg_ha', 'K_kg_ha']:
                    if nutrient in group.columns:
                        values = group.loc[non_icv_mask, nutrient]
                        weights_subset = group.loc[non_icv_mask, 'surface_travaillee_ha']
                        if not values.empty and not weights_subset.empty:
                            result[f"non_icv_{nutrient}"] = np.average(
                                values,
                                weights=weights_subset
                            )
            
            return pd.Series(result)


        
        # Grouper par SIRET et culture
        result = df.groupby(['siret_exploitation', 'culture']).apply(weighted_average).reset_index()
        
        print(f"\nRésultat de l'agrégation:")
        print(f"Nombre de combinaisons exploitation-culture: {len(result)}")
        print(f"Nombre d'exploitations: {result['siret_exploitation'].nunique()}")
        print(f"Nombre de cultures: {result['culture'].nunique()}")
        
        return result

    def combine_sources(self, mesparcelles_data, smag_data, smag_cultures_data):
        """Combine les données des deux sources"""
        print("Combinaison des données...")
        
        # Standardisation des données
        print("\nTraitement des données MesParcelles...")
        mp_std = self.standardize_mesparcelles(mesparcelles_data)
        
        print("\nTraitement des données SMAG...")
        smag_std = self.standardize_smag(smag_data, smag_cultures_data)
        
        if mp_std is None or smag_std is None:
            print("ERREUR: problème dans la standardisation des données")
            return None
        
        # Ajout de la source
        mp_std['source'] = 'mesparcelles'
        smag_std['source'] = 'smag'
        
        # Combinaison des données
        combined = pd.concat([mp_std, smag_std], ignore_index=True)
        
        print("\nRésumé des données combinées:")
        print(f"Nombre total d'observations: {len(combined)}")
        print(f"Nombre d'exploitations uniques: {combined['siret_exploitation'].nunique()}")
        print(f"Nombre de cultures uniques: {combined['culture'].nunique()}")
        print(f"Cultures uniques trouvées: {combined['culture'].unique()}")
        
        return combined

# Code d'utilisation
if __name__ == "__main__":
    # Import des données
    print("Import des données...")
    
    # Import MesParcelles
    mp_path = r"C:\Users\MonirNajem\OneDrive - FOOD PILOT\Desktop\monir\MESPARCELLES\mesparcelles_data.xlsx"
    mp_data = pd.read_excel(mp_path, sheet_name='Intervention_Details')
    print(f"Données MesParcelles importées : {len(mp_data)} lignes")
    
    # Import SMAG
    smag_path = r"C:\Users\MonirNajem\OneDrive - FOOD PILOT\Desktop\monir\MESPARCELLES\SMAG.xlsx"
    smag_data = pd.read_excel(smag_path)
    print(f"Données SMAG importées : {len(smag_data)} lignes")
    
    # Import SMAG Cultures
    smag_cultures_path = r"C:\Users\MonirNajem\OneDrive - FOOD PILOT\Desktop\monir\MESPARCELLES\SMAG_cultures.xlsx"
    smag_cultures = pd.read_excel(smag_cultures_path)
    print(f"Données cultures SMAG importées : {len(smag_cultures)} lignes")
    
    # Traitement des données
    standardizer = FertilizerStandardizer()
    combined_data = standardizer.combine_sources(mp_data, smag_data, smag_cultures)
    
    # Export des résultats
    if combined_data is not None:
        output_path = 'resultats_standardisation_par_exploitation_culture.xlsx'
        combined_data.to_excel(output_path, index=False)
        print(f"\nRésultats exportés dans '{output_path}'")
        
        # Afficher quelques statistiques finales
        print("\nRésumé final :")
        print(f"Nombre total de combinaisons exploitation-culture: {len(combined_data)}")
        print(f"Nombre d'exploitations: {combined_data['siret_exploitation'].nunique()}")
        print(f"Nombre de cultures: {combined_data['culture'].nunique()}")
        print("\nNombre d'observations par source:")
        print(combined_data['source'].value_counts())

Import des données...
Données MesParcelles importées : 14 lignes
Données SMAG importées : 34540 lignes
Données cultures SMAG importées : 9311 lignes
Combinaison des données...

Traitement des données MesParcelles...
Standardisation des données MesParcelles...
Colonne culture.libelle trouvée et renommée en 'culture'
Cultures uniques dans MesParcelles: ['blé tendre hiver' 'blé dur printemps']

Après standardisation MesParcelles:
Nombre d'observations: 8
Nombre d'exploitations: 1
Nombre de cultures: 2

Agrégation par exploitation et culture...

Résultat de l'agrégation:
Nombre de combinaisons exploitation-culture: 10
Nombre d'exploitations: 1
Nombre de cultures: 2

Traitement des données SMAG...

Standardisation des données SMAG...

Avant fusion - Nombre de lignes: 19547


C:\Users\MonirNajem\AppData\Local\Temp\ipykernel_25020\2920270959.py:261: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby(['siret_exploitation', 'culture']).apply(weighted_average).reset_index()


Après fusion - Nombre de lignes: 2498226
Lignes avec culture non-définie: 13135

Après standardisation SMAG:
Nombre d'observations: 2498226
Nombre d'exploitations: 201
Nombre de cultures: 95

Agrégation par exploitation et culture...


C:\Users\MonirNajem\AppData\Local\Temp\ipykernel_25020\2920270959.py:261: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby(['siret_exploitation', 'culture']).apply(weighted_average).reset_index()



Résultat de l'agrégation:
Nombre de combinaisons exploitation-culture: 11047
Nombre d'exploitations: 201
Nombre de cultures: 95

Résumé des données combinées:
Nombre total d'observations: 11057
Nombre d'exploitations uniques: 202
Nombre de cultures uniques: 97
Cultures uniques trouvées: ['blé dur printemps' 'blé tendre hiver' 'Bandes tampons/enherbées'
 'Blé tendre hiver' 'Bois pâturé' 'Féverole hiver' 'Jachère autre semée'
 'Maïs doux' 'Maïs fourrage' 'Maïs grain' 'Maïs waxy'
 'Mélange légum./gram. fourrage' 'Pomme de terre chair ferme'
 'Pomme de terre consom.' 'Prairie permanente pâtur./fauch.'
 'Prairie temporaire fauchée' 'Prairie temporaire pâtur./fauch.'
 'Prairie à rotation longue pâtur./fauch.' 'Soja' 'Sorgho grain'
 'Surface non exploitée' 'Tournesol' 'Tournesol semence' 'Vigne'
 "Arbres d'ornement" 'Autres utilisations' 'Avoine noire hiver'
 'Bandes tampons/enherbées prairie' 'Blé dur hiver' 'Bordure de champ'
 'Colza oléagineux hiver' 'Colza oléagineux hiver semence'
 'Fét